# Stable Baseline 3

In [92]:
import sys

path_up = ".."
if not path_up in sys.path:
    sys.path.append(path_up)

In [93]:
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor

from funcs.ios import get_excel_sheet
from modules.env import TraningEnv

In [94]:
path_excel = "../collection/ticks_20250925.xlsx"
code = "7011"
df = get_excel_sheet(path_excel, code)
df

,Time,Price,Volume
0,1.758758e+09,3881,1046300
1,1.758758e+09,3881,1046300
2,1.758758e+09,3870,1192700
3,1.758758e+09,3870,1192700
4,1.758758e+09,3876,1198000
...,...,...,...
19377,1.758781e+09,3850,27825200
19378,1.758781e+09,3850,27825200
19379,1.758781e+09,3850,27826300
19380,1.758781e+09,3850,27826300


In [95]:
log_dir = "./logs/"
env = TraningEnv(df)
# env = Monitor(env, log_dir, allow_early_resets=False)  # Monitorの利用
model = PPO("MlpPolicy", env, verbose=True)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [89]:
model.learn(total_timesteps=128000)

-----------------------------
| time/              |      |
|    fps             | 1710 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1378        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013424433 |
|    clip_fraction        | 0.225       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.37       |
|    explained_variance   | 2.26e-06    |
|    learning_rate        | 0.0003      |
|    loss                 | 30.7        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0267     |
|    value_loss           | 120         |
-----------------------------------------
----------------------------------

In [90]:
obs, info = env.reset()
total_reward = 0.0
total_pnl = 0.0

# 推論の実行
while True:
    # モデルの推論
    arr_action, _states = model.predict(obs, deterministic=True)
    action = arr_action.item()

    # 1ステップ実行
    obs, reward, done, truncated, info = env.step(action)

    # モデル報酬
    total_reward += reward

    # エピソード完了
    if done:
        break

# 学習環境の解放
env.close()

In [91]:
# モデル報酬（総額）
print(f"モデル報酬（総額）: {total_reward:.2f}")

print("取引明細")
print(pd.DataFrame(env.tamer.dict_transaction))
if "pnl_total" in info.keys():
    print(f"--- テスト結果 ---")
    print(f"最終的な累積報酬（利益）: {info['pnl_total']:.2f}")

モデル報酬（総額）: 973.04
取引明細
                      注文日時 銘柄コード  売買  約定単価  約定数量   損益
0      2025-09-25 09:01:09  7011  売建  3892     1  NaN
1      2025-09-25 09:01:10  7011  買埋  3890     1  2.0
2      2025-09-25 09:01:11  7011  売建  3890     1  NaN
3      2025-09-25 09:01:12  7011  買埋  3891     1 -1.0
4      2025-09-25 09:01:13  7011  売建  3891     1  NaN
...                    ...   ...  ..   ...   ...  ...
19317  2025-09-25 15:24:55  7011  買埋  3850     1  0.0
19318  2025-09-25 15:24:56  7011  売建  3850     1  NaN
19319  2025-09-25 15:24:57  7011  買埋  3850     1  0.0
19320  2025-09-25 15:24:58  7011  売建  3850     1  NaN
19321  2025-09-25 15:24:59  7011  買埋  3850     1  0.0

[19322 rows x 6 columns]
--- テスト結果 ---
最終的な累積報酬（利益）: 7.00
